In [1]:
import os, psutil, gc
import time 
import json
import pprint

from collections import defaultdict
import random

import numpy as np
from scipy.stats import ttest_rel

In [2]:
from sal.config import Config

from core import best_of_n
from utils.load_data import load_data_prm800k
from utils import grader 

In [3]:
# base_dir
base_dir = '/groups/kjun/tnn/datasets/'

# dataset path
data_dir = base_dir + "/prm800k/math_splits"

# llm and prm path
llm_dir = base_dir + "/Llama-3.2-1B-Instruct-GGUF/Llama-3.2-1B-Instruct.Q4_K_M.gguf"
prm_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data-GGUF/Llama3.1-8B-PRM-Deepseek-Data.Q4_K_M.gguf"

llm_tokenizer_dir = base_dir + "/Llama-3.2-1B-Instruct"
prm_tokenizer_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data"

In [4]:
#  load data 
data_by_levels = load_data_prm800k(data_dir)


# ds_completions = load_completions(completions_dir)

# load random_seeds     
# random_seeds = np.loadtxt("random_seeds.txt").astype("int64")
# random_seeds = [int(seed) for seed in random_seeds]

1: 43
2: 90
3: 105
4: 128
5: 134


In [5]:
level = '4'
num_questions = len(data_by_levels[level])
# num_questions = 1
num_trials = 1
num_budgets = 8
print(f"num_questions = {num_questions}")

num_questions = 128


In [10]:
def evaluate_correctness(data_dir, data_by_levels, num_trials, num_budgets):
    with open(data_dir, 'r', encoding='utf-8') as fin:
        results = []
        best_completions = []
        best_prm_scores = []
        pred_answers = []
        gt_answers = []
        trial_idx = 0
        for line in fin:
            if trial_idx >= num_trials:
                break
                
            trial_data = json.loads(line)
            for q_idx in range(len(data_by_levels)):
                best_idx = np.argmax(trial_data['agg_scores'][q_idx])
                best_completion = trial_data['completions'][q_idx][best_idx]
                pred_answer = grader.extract_last_boxed_answer(best_completion)
                gt_answer = data_by_levels[q_idx]['answer']
                is_correct = grader.grade_answer(pred_answer, gt_answer)

                # # print(len(trial_data['completions'][q_idx]))
                # if pred_answer is None and q_idx > 1:
                #     print(f"\n-> question {q_idx}")
                #     print(f"pred answer: {pred_answer}")
                #     print(f"gt answer: {gt_answer}")
                #     print(f"is correct: {is_correct}")
                #     # print(best_completion)
                #     print(best_idx)
                #     for cidx, completion in enumerate(trial_data['completions'][q_idx]):
                #         print(f"cidx = {cidx}")
                #         print(completion)
                # print(f"all scores = {trial_data['agg_scores'][q_idx]}")
                # print(f"best score = {trial_data['agg_scores'][q_idx][best_idx]}")
                results.append(is_correct)
                best_completions.append(best_completion)
                pred_answers.append(pred_answer)
                gt_answers.append(gt_answer)
                best_prm_scores.append(trial_data['agg_scores'][q_idx][best_idx])

        trial_idx += 1
        
    return results, best_completions, best_prm_scores, pred_answers, gt_answers


# general params
config = Config()
config.agg_strategy = 'last'
config.n = 16
config.beam_width = 2
config.lookahead = 0
config.num_iterations = 3
config.sort_completed = False

# diverse_select params
config.lam = 10
config.normalize_embeds = True

bon_dir = "results/scores_bon_prm800k_level4_n16_v11.jsonl" 
sd_dir = f"results/scores_sd_prm800k_level{level}_n{config.n}_bw{config.beam_width}_depth{config.num_iterations}_lam{config.lam}_v11.jsonl"

bon_results, bon_best_completions, bon_best_prm_scores, bon_pred_answers, bon_gt_answer = \
    evaluate_correctness(bon_dir, data_by_levels[level], num_trials, config.n)
sd_results, sd_best_completions, sd_best_prm_scores, sd_pred_answers, sd_gt_answer = \
    evaluate_correctness(sd_dir, data_by_levels[level], num_trials, config.n)
# print(sd_results)
# print(sd_best_completions)
# print(sd_pred_answers)
# print(sd_gt_answer)
print(f"bon_score = {np.mean(bon_results)}")
print(f"sd_score = {np.mean(sd_results)}")

t_stat, p_value = ttest_rel(np.array(bon_results).astype(int), np.array(sd_results).astype(int))
print(t_stat)
print(p_value)

bon_score = 0.328125
sd_score = 0.3359375
-0.20773359138114222
0.8357697355343734
